In [111]:
from __future__ import print_function

import sys
from operator import add
from pyspark.sql import SparkSession

import re

def createNameMap(nameList):
    nameMap = {}
    for name in nameList:
        nameMap[name[0]] = name[1]
    return nameMap

def append(x, y):
    for item in x:
        y.append(item)
    return y
    
def mapreduce(inPath, outPath, spark):
    lines  = spark.read.text(inPath) \
            .rdd.map(lambda x:x[0]) \
            .map(lambda x:x.split(',')).map(lambda x:[x[0], [x[1], int(x[2])]])
    summary = lines.map(lambda x: [x[0], x[1][1]]).reduceByKey(add)
    nameMap = createNameMap(summary.collect())
    
    weight = lines.map(lambda x: [x[0], [[x[1][0], x[1][1]/float(nameMap[x[0]])]]]) \
            .reduceByKey(append) #将同一人物的相关人物放到一起
    output = weight.collect()
    
    fout = open(outPath, 'w', encoding='utf-8')
    for (key, valueList) in output:
        fout.write(key + '\t')
        for value in valueList:
            fout.write(value[0] + ',' + str(value[1]) + '|')
        fout.write('\n')
    fout.close()
    
def drive():
    spark = SparkSession\
    .builder\
    .appName("Task3")\
    .getOrCreate()
    mapreduce('./../result/task2.out', './../result/task3.out', spark)
drive()

143


[['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['戚芳', ['戚长发', 0.25]], ['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['卜垣', ['戚长发', 0.25]], ['戚芳', ['戚长发', 0.25]], ['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['戚长发', ['卜垣', 0.3333333333333333]], ['卜垣', ['戚长发', 0.25]], ['戚芳', ['戚长发', 0.25]], ['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['戚长发', ['狄云', 0.3333333333333333]], ['戚长发', ['卜垣', 0.3333333333333333]], ['卜垣', ['戚长发', 0.25]], ['戚芳', ['戚长发', 0.25]], ['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['戚芳', ['卜垣', 0.5]], ['戚长发', ['狄云', 0.3333333333333333]], ['戚长发', ['卜垣', 0.3333333333333333]], ['卜垣', ['戚长发', 0.25]], ['戚芳', ['戚长发', 0.25]], ['卜垣', ['狄云', 0.25]], ['卜垣', ['戚芳', 0.5]], ['狄云', ['戚芳', 0.3333333333333333]]]
[['戚芳', ['狄云', 0.25]], ['戚芳', ['卜垣', 0.5]], ['戚长发